# Métrica para clusterização

Para medir a semelhança entre o processamento dinâmico de diferentes páginas, parece razoável comparar os botões presentes em cada uma dessas páginas. Para se fazer isso, são necessárias as seguintes bibliotecas:

In [6]:
import urllib3
import jellyfish
import requests
import numpy as np
import asyncio

from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib3.exceptions import *
from html_similarity import style_similarity, structural_similarity, similarity

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from playwright.async_api import async_playwright

Definindo as opções de inicialização do selenium:

In [7]:
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-web-security")
options.add_argument('--headless')

Há duas funções para extração de botões de uma página. A princípio, utilizará-se a primeira.

In [8]:
def extrair_botoes_selenium(url):

    resultados = ""

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')
    buttons = soup.find_all('button')
    for button in buttons:
        resultados+=str(button)+"\n"
        
    return resultados

In [9]:
async def extrair_botoes_playwright(url):
    
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = True)
    page = await browser.new_page()
    await page.goto(url)
    
    resultados = ""

    try:
        html = await page.content()
        
    except:
        await page.close()
        await browser.close()
        
        return "Error: " + url_page
    
    soup = BeautifulSoup(html, 'html.parser')
    buttons = soup.find_all('button')
    for button in buttons:
        resultados+=str(button)+"\n"
        
    await page.close()
    await browser.close()
        
    return resultados

A função a seguir gera um resultado de similaridade entre as duas instâncias de botões passadas.

In [11]:
def gerar_resultado(botoes_1, botoes_2):
    
    return similarity(botoes_1, botoes_2)

Faz-se necessário transformar um conjunto de urls para um conjunto de botões. Pode ser vista como uma etapa de pré-processamento.

In [28]:
def links_para_botoes(urls):
    
    botoes = []
    
    for url in urls:

        if url != "":
            
            try:
                resultado = extrair_botoes_selenium(url)
                botoes.append(resultado)

            except Exception as e:
                print(e)
                
        else:
            continue
        
    arq_botoes = open("botoes.txt", "a")
    arq_botoes.write(str(botoes))
    arq_botoes.close()

Por fim, a última função gera a matriz de distâncias diretamente dos dados produzidos. Essa matriz será simétrica.

In [1]:
def gerar_matriz_distancias(dados):
    
    tam = len(dados)
    resultados = np.ones((tam, tam), dtype = np.float32)
    
    for i in range(0, tam):
        for j in range(0, i+1):
            similaridade = gerar_resultado(dados[i], dados[j])
            if i != j:
                resultados[i,j] -= similaridade
            resultados[j,i] -= similaridade
            
            
    return resultados
        